In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp

## Solve for $\theta^*$

In [2]:
### First, estimate theta*, true causal effect of (SAT, HS GPA) on college GPA
## based on real data from 1000 students
df = pd.read_csv("clean_gpa.csv")

x_real = df[['new_sat','new_gpa']].to_numpy()
y_real = df['college_gpa'].to_numpy()

## find true causal effects theta*
# ordinary least squares (ols)
x_tilde = np.hstack((x_real,np.ones((len(x_real),1)))) # add 1 for intercept

m = x_real.shape[1]
x_sum = np.zeros([m+1,m+1])
xy_sum = np.zeros(m+1)

for i in range(len(x_real)):
  x_sum += np.outer(x_tilde[i],x_tilde[i])
  xy_sum += x_tilde[i]*y_real[i]

theta_star = np.matmul(np.linalg.inv(x_sum),xy_sum)[:-1]

# set theta* to nice values for synthetic data
theta_star = np.array([0,0.5])

## Agent outcomes

In [3]:
def ols(x,y,T): # with intercept estimation
  x_tilde = np.hstack((x,np.ones((len(x),1)))) # for parameter estimation

  m = x.shape[1]
  x_sum = np.zeros([m+1,m+1])
  xy_sum = np.zeros(m+1)

  for i in range(T):
    x_sum += np.outer(x_tilde[i],x_tilde[i])
    xy_sum += x_tilde[i]*y[i]

  theta_hat_ols = np.matmul(np.linalg.inv(x_sum),xy_sum)
  return theta_hat_ols[:m]

def tsls(x,y,theta,T): # runs until round T
  theta_tilde = np.hstack((theta,np.ones((len(theta),1)))) # for parameter estimation

  m = x.shape[1]
  theta_tilde_sum = np.zeros([m+1,m+1])
  xtheta_tilde_sum = np.zeros([m+1,m])
  ytheta_tilde_sum = np.zeros(m+1)

  for i in range(T):
    theta_tilde_sum += np.outer(theta_tilde[i],theta_tilde[i])
    xtheta_tilde_sum += np.outer(theta_tilde[i],x[i])
    ytheta_tilde_sum += theta_tilde[i]*y[i]

  # Step 1) estimate Omega: regress theta onto x
  omega_hat = np.matmul(np.linalg.inv(theta_tilde_sum),xtheta_tilde_sum)
  z_bar = omega_hat[m,:]
  omega_hat = omega_hat[:m,:m] 

  # Step 2) estimate Lambda: regress theta onto y
  lambda_hat = np.matmul(np.linalg.inv(theta_tilde_sum),ytheta_tilde_sum)
  gztheta_bar = lambda_hat[m]
  lambda_hat = lambda_hat[:m]

  # Step 3) estimate theta*: inverse(Omega-hat)*Lambda-hat
  theta_hat_tsls = np.matmul(np.linalg.inv(omega_hat),lambda_hat)
  return theta_hat_tsls

In [4]:
num_applicants = 1000
half = int(num_applicants/2) 

theta_star = np.array([0, 0.5])
m = theta_star.size

sigma_g = 0.1 # g variance term
mean_sat = 900
mean_gpa = 2
sigma_sat = 200
sigma_gpa = 0.5

# initial features (z)
z = np.zeros([num_applicants,m])

# disadvantaged students
z[0:half,0] = np.random.normal(mean_sat-100,sigma_sat,z[0:half,0].shape) #SAT
z[0:half,1] = np.random.normal(mean_gpa-.2,sigma_gpa,z[0:half,1].shape) #GPA

# advantaged students
z[half:,0] = np.random.normal(mean_sat+100,sigma_sat,z[0:half,0].shape) #SAT
z[half:,1] = z[half:,1] + np.random.normal(mean_gpa+.2,sigma_gpa,z[half:,1].shape) #GPA

z[:,0] = np.clip(z[:,0],400,1600) # clip to 400 to 1600
z[:,1] = np.clip(z[:,1],0,4) # clip to 0 to 4.0

# confounding error term g (error on true college GPA)
g = np.ones(num_applicants)*0.5 # legacy students shifted up
g[0:half]=-0.5 # first-gen students shifted down
g += np.random.normal(1,0.2,size=num_applicants) # non-zero-mean

# assessment rule 
theta = np.zeros([num_applicants,z.shape[1]])
theta = np.random.normal(1,1,[num_applicants,z.shape[1]])
theta[:,0]*=7.5 # scaling for SAT score

# expected effort conversion matrices E[WW^T]
EWWT = np.matrix([[5,0.05],[0.01,0.4]])

# effort conversion matrices W_t*W_t^T
WWT = list()

for i in range(num_applicants):
  WWT_t = EWWT.copy()

  # add / subtract noise to E[WW^T]
  noise00 = np.random.normal(EWWT[0,0]/10,EWWT[0,0]/10)
  noise01 = np.random.normal(EWWT[0,1]/10,EWWT[0,1]/10)
  noise10 = np.random.normal(EWWT[1,0]/10,EWWT[1,0]/10)
  noise11 = np.random.normal(EWWT[1,1]/10,EWWT[1,1]/10)
  noise = np.array([[noise00,noise01],[noise10,noise11]])

  if i<half: # first-gen
    WWT_t -= noise/2
  else: # legacy
    noise[0,0]*=7.5
    WWT_t += noise*2

  WWT.append(WWT_t)
WWT = np.array(WWT)

# observable features x
x = np.zeros([num_applicants,z.shape[1]])
for i in range(num_applicants):
  x[i] = z[i] + np.matmul(WWT[i],theta[i])

x[:,0] = np.clip(x[:,0],400,1600) # clip to 400 to 1600
x[:,1] = np.clip(x[:,1],0,4) # clip to 0 to 4.0

# true outcomes (college gpa)
y = np.clip(np.matmul(x,theta_star) + g,0,4)

T = len(x)
ols_estimate = ols(x, y, T) # ols w/ intercept estimate
tsls_estimate = tsls(x, y, theta, T) # 2sls w/ intercept estimate

In [5]:
## solve for true Lambda and estimated Lambda
# regress theta_t onto y (with centering)
m = x.shape[1]
theta_sum = np.zeros([m,m])
ytheta_sum = np.zeros(m)
for i in range(T):
  theta_centered = theta[i]-np.mean(theta[:T],axis=0)
  theta_sum += np.outer(theta_centered,theta_centered)
  ytheta_sum += theta_centered*(y[i]-np.mean(y[:T]))

Lambda_hat = np.matmul(np.linalg.inv(theta_sum),ytheta_sum)
print(Lambda_hat)

Lambda = theta_star*EWWT
Lambda = np.array([Lambda[0,0],Lambda[0,1]]) # recast into vector
print(Lambda)

print(np.linalg.norm(Lambda_hat-Lambda))

[0.00210626 0.24546243]
[0.005 0.2  ]
0.045554427111434516


In [6]:
## solve for AO maximizing theta (theta_ao)

# solve for true Lambda and estimated Lambda
# by regressing theta_t onto y (with centering)
m = x.shape[1]
theta_sum = np.zeros([m,m])
ytheta_sum = np.zeros(m)
for i in range(T):
  theta_centered = theta[i]-np.mean(theta[:T],axis=0)
  theta_sum += np.outer(theta_centered,theta_centered)
  ytheta_sum += theta_centered*(y[i]-np.mean(y[:T]))

Lambda_hat = np.matmul(np.linalg.inv(theta_sum),ytheta_sum)
print(Lambda_hat)

Lambda = np.matmul(EWWT,theta_star)
Lambda = np.array([Lambda[0,0],Lambda[0,1]]) # recast into vector

# solve LP with Lambda and its estimate (Lambda_hat)
c1 = [-Lambda[0], -Lambda[1]]
c2 = [-Lambda_hat[0], -Lambda_hat[1]]
A = [[1, 1]]
b = [10]
theta0_bounds = (-30, 30)
theta1_bounds = (-3, 3)
from scipy.optimize import linprog
opt1 = linprog(c1, A_ub=A, b_ub=b, bounds=[theta0_bounds, theta1_bounds])
opt2 = linprog(c2, A_ub=A, b_ub=b, bounds=[theta0_bounds, theta1_bounds])

theta_ao = opt1.x
theta_ao_hat = opt2.x

[0.00210626 0.24546243]


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [7]:
import cvxpy as cp

# Solve for true agent outcomes-maximizing theta_ao
theta_ao = cp.Variable((len(Lambda)))

# Create two constraints.
constraints = [theta_ao[0] >= -30, theta_ao[0] <= 30,
               theta_ao[1] >= -3, theta_ao[1] <= 3,
               cp.norm(theta_ao) <= 10]

# Form objective.
obj = cp.Maximize(cp.matmul(theta_ao,Lambda))

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.

0.8384848001803566

In [8]:
# Solve for estimated agent outcomes-maximizing theta_ao_hat 
# (using estimate theta_star_hat
theta_ao_hat = cp.Variable((len(Lambda_hat)))

# Create two constraints.
constraints = [theta_ao_hat[0] >= -30, theta_ao_hat[0] <= 30,
               theta_ao_hat[1] >= -3, theta_ao_hat[1] <= 3,
               cp.norm(theta_ao_hat) <= 10]

# Form objective.
obj = cp.Maximize(cp.matmul(theta_ao_hat,Lambda_hat))

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.

0.756479715128524

In [9]:
# agent outcomes for different theta
theta_ols = ols(x,y,len(x)) # ols estimate of theta

# initialize lists of x's
x_star = np.zeros([num_applicants,z.shape[1]])
x_ols = np.zeros([num_applicants,z.shape[1]])
x_ao = np.zeros([num_applicants,z.shape[1]])
x_ao_hat = np.zeros([num_applicants,z.shape[1]])

for i in range(num_applicants):
  x_star[i] = z[i] + np.matmul(WWT[i],theta_star)
  x_ols[i] = z[i] + np.matmul(WWT[i],theta_ols)
  x_ao[i] = z[i] + np.matmul(WWT[i],theta_ao.value)
  x_ao_hat[i] = z[i] + np.matmul(WWT[i],theta_ao_hat.value)

# clip x values
def clip_x(x):
  x[:,0] = np.clip(x[:,0],400,1600) # clip to [400, 1600]
  x[:,1] = np.clip(x[:,1],0,4) # clip to [0, 4]
  return x

x_star = clip_x(x_star)
x_ols = clip_x(x_ols)
x_ao = clip_x(x_ao)
x_ao_hat = clip_x(x_ao_hat)

# true outcomes (college gpa)
y_star = np.clip(np.matmul(x_star,theta_star) + g, 0, 4)
y_ols = np.clip(np.matmul(x_ols,theta_star) + g, 0, 4)
y_ao = np.clip(np.matmul(x_ao,theta_star) + g, 0, 4)
y_ao_hat = np.clip(np.matmul(x_ao_hat,theta_star) + g, 0, 4)

print("Mean reward playing theta*:", np.mean(y_star))
print("Mean reward playing theta_hat (OLS):",np.mean(y_ols))
print("Mean reward playing theta_ao (w/ true Lambda):",np.mean(y_ao))
print("Mean reward playing theta_ao_hat (w/ estimated Lambda_hat):",np.mean(y_ao_hat))

Mean reward playing theta*: 2.1028671413250057
Mean reward playing theta_hat (OLS): 2.154948137128592
Mean reward playing theta_ao (w/ true Lambda): 2.6594687137431166
Mean reward playing theta_ao_hat (w/ estimated Lambda_hat): 2.6594687137567434
